# tourapi

In [1]:
## touristattractions 

In [1]:
import pandas as pd
tourapi = pd.read_excel('/home/hwang/projects/yanolja/data/dataset/tourAPI/touristAttractions.xls')

In [2]:
tourapi['상세정보'][0]

'이용가능시설:- 수성못, 오리배&유람선, 수변데크로드, 포켓무대, 관광안내소 북카페, 상화동산, 시문학거리, 들안길 시화거리, 영상음악분수, 아르떼 수성랜드, 수성파크랜드<br />\r\n<br />\r\n※ 기상여건 및 사정에따라 이용불가시설 있음\n화장실:있음\n'

In [3]:
tourapi['source'] = 'tourAPI'

In [4]:
tourapi['category'] = "tourist_attractions"

In [5]:
tourapi.head(2)

,명칭,우편번호,관리자,전화번호,주소,위도,경도,개요,유산구분,문의 및 안내,...,수용인원,이용시기,이용시간,주차시설,유모차 대여 여부,애완동물 동반 가능 여부,신용카드 가능 여부,상세정보,source,category
0,수성못 유원지,42202,NaN,NaN,대구광역시 수성구 용학로 35-5,35.826842,128.612873,수성유원지는 범물동 용지봉(629ｍ)에서 북서부로 뻗어 내린 줄기의 하부에 위치하여...,NaN,수성유원지 053-666-2863<br/>\n수성못 관광안내소 053-761-0645,...,NaN,NaN,00:00~24:00<br>\r\n※ 수성 미디어아트 음악분수 운영시간은 홈페이지 ...,가능,없음,NaN,NaN,"이용가능시설:- 수성못, 오리배&유람선, 수변데크로드, 포켓무대, 관광안내소 북카페...",tourAPI,tourist_attractions
1,동락공원,39392,NaN,NaN,경상북도 구미시 3공단1로 191(진평동),36.097116,128.401951,낙동강을 따라 구미대교 아래 임수동에서 칠곡군 석적읍 중리까지 이르는 수변형 도시공...,NaN,054-480-4612,...,NaN,NaN,상시개방,있음(약 소형 946대),없음,NaN,NaN,주차요금:무료\n입 장 료:무료\n화장실:있음(남/녀 구분)\n이용가능시설:* 체육...,tourAPI,tourist_attractions


In [6]:
tourapi.columns

Index(['명칭', '우편번호', '관리자', '전화번호', '주소', '위도', '경도', '개요', ' 유산구분', '문의 및 안내',
       '개장일', '쉬는날', '체험안내', '체험가능연령', '수용인원', '이용시기', '이용시간', '주차시설',
       '유모차 대여 여부', '애완동물 동반 가능 여부', '신용카드 가능 여부', '상세정보', 'source',
       'category'],
      dtype='object')

In [7]:
tourapi.rename(columns= {'명칭': 'place_name', 
                                   '주소':'address', 
                                   '위도': 'latitude', 
                                   '경도': 'longitude', 
                                   '이용시간': 'available_time', 
                                   '주차시설': 'parking', 
                                   '개요': 'description', 
                                   '우편번호': 'zip_code', 
                                   '관리자': 'manager',
                                   '전화번호': 'phone',
                                   '문의 및 안내': 'guidance',
                                   '수용인원': 'capacity',
                                   '개장일': 'opening_date', ##
                                   '쉬는날': 'day_off',
                                   '이용시기': 'available_period', ## 
                                   ' 유산구분': 'heritage', ##
                                   '체험안내': 'experience_guide', ##
                                   '체험가능연령': 'age_available', ##
                                   '유모차 대여 여부': 'stroller',
                                   '애완동물 동반 가능 여부': 'pet',
                                   '신용카드 가능 여부': 'credit_card',
                                   '상세정보': 'details'}, inplace=True)

tourapi = tourapi[['source', 'category', 'place_name', 'address', 'latitude', 'longitude', 'available_time', 'parking', 'description', 'zip_code', 'manager', 'phone', 'guidance', 'capacity', 'day_off', 
                   'opening_date', 'available_period', 'heritage', 'experience_guide', 'age_available', 'stroller', 'pet', 'credit_card', 'details']]

In [8]:
tourapi.isnull().sum() / len(tourapi)

source              0.000000
category            0.000000
place_name          0.000000
address             0.000077
latitude            0.000000
longitude           0.000000
available_time      0.335860
parking             0.241004
description         0.001072
zip_code            0.038432
manager             0.973358
phone               0.974812
guidance            0.029169
capacity            0.908973
day_off             0.358751
opening_date        0.924055
available_period    0.984612
heritage            0.995100
experience_guide    0.770096
age_available       0.870540
stroller            0.133670
pet                 0.441510
credit_card         1.000000
details             0.242765
dtype: float64

In [9]:
tourapi.isnull().sum()

source                  0
category                0
place_name              0
address                 1
latitude                0
longitude               0
available_time       4387
parking              3148
description            14
zip_code              502
manager             12714
phone               12733
guidance              381
capacity            11873
day_off              4686
opening_date        12070
available_period    12861
heritage            12998
experience_guide    10059
age_available       11371
stroller             1746
pet                  5767
credit_card         13062
details              3171
dtype: int64

In [10]:
import openai
import pandas as pd
import re
import json
import os
from tqdm import tqdm
from dotenv import load_dotenv

if not load_dotenv(dotenv_path="/home/hwang/projects/yanolja/src/.env"):
    print(".env 파일을 로드하지 못했습니다. 경로를 확인하세요.")
openai.api_key = os.getenv("OPENAI_API_KEY")
if openai.api_key is None:
    raise ValueError("OpenAI API 키가 설정되지 않았습니다. .env 파일을 확인하세요.")

df = tourapi[7000:]

def preprocess_text(text):
    if pd.isna(text) or text.strip() == "":
        return "정보 없음"
    text = re.sub(r"<[^>]+>", " ", text)  
    text = re.sub(r"\s+", " ", text).strip()  
    return text

df["day_off"] = df["day_off"].apply(preprocess_text)
df["available_time"] = df["available_time"].apply(preprocess_text)
df["available_period"] = df["available_period"].apply(preprocess_text)

def transform_to_standard_format(day_off, available_time, available_period):
    day_off = preprocess_text(str(day_off))
    available_time = preprocess_text(str(available_time))
    available_period = preprocess_text(str(available_period))
    
    prompt = f"""

    **입력 데이터**:
    - 영업기간(available_period): {available_period}
    - 영업시간(available_time): {available_time}
    - 휴무일(day_off): {day_off}

    입력 데이터에 대해서 이해한 바를 창조하지 말고 사실에 기반하여 서술하세요.
    """
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0,
        max_tokens=500
    )
    content = response['choices'][0]['message']['content']
    return content


results = []
for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing rows"):
    transformed = transform_to_standard_format(row["day_off"], row["available_time"], row["available_period"])
    results.append({"row_index": index, "transformed_data": transformed})

output_file = "transformed_tourapi_touristattractions_available_time_results_v1_2.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f"변환된 결과가 {output_file} 파일에 저장되었습니다.")


/tmp/ipykernel_431131/1521089742.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["day_off"] = df["day_off"].apply(preprocess_text)
/tmp/ipykernel_431131/1521089742.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["available_time"] = df["available_time"].apply(preprocess_text)
/tmp/ipykernel_431131/1521089742.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

변환된 결과가 transformed_tourapi_touristattractions_available_time_results_v1_2.json 파일에 저장되었습니다.


In [11]:
import json
import openai

import os
from dotenv import load_dotenv

from tqdm import tqdm

load_dotenv(dotenv_path="/home/hwang/projects/yanolja/src/.env")
openai.api_key = os.getenv("OPENAI_API_KEY")

input_file = "/home/hwang/projects/yanolja/src/collecting_data/data_analysis/transformed_tourapi_touristattractions_available_time_results_v1_2.json"
output_file = "transformed_tourapi_touristattractions_available_time_results_v2_2.json"

def transform_data_with_gpt(input_text):
    system_prompt = (
        "You are an assistant that transforms text about business hours and holidays into a structured JSON format. "
        "The JSON must strictly adhere to the following rules and reflect only the information explicitly provided in the input text. "
        "If the input specifies different 'entry closing time' (e.g., last entry allowed) and 'closing time' (e.g., business hours end), these must be clearly separated:\n\n"
        "Rules for transformation:\n"
        "1. The 'hours' key should only include the official business hours for each day (Monday to Sunday).\n"
        "   - If no business hours are provided for a specific day, leave that day as an empty list (e.g., \"Monday\": []).\n"
        "   - 'Closing time' refers to the time when the business officially stops operating and should be used in the 'hours' key.\n\n"
        "2. The 'special_conditions' key should include:\n"
        "   - 'entry closing time' (last entry allowed) as a separate condition with 'type: \"entry_closing_time\"' if explicitly stated.\n"
        "   - Any holidays, seasonal hours, or other exceptions.\n"
        "   - Each entry must have a 'type' and 'details'.\n\n"
        "3. If conflicting or ambiguous information is present in the input, separate the conflicting details into individual entries in 'special_conditions'.\n\n"
        "4. Do not infer or assume any information. Only use the data explicitly provided in the input text.\n"
        "   - If any information is missing or incomplete, leave the corresponding field empty (e.g., `[]`, `{}`).\n"
        "   - Only add 'details: \"Information not provided\"' if explicitly mentioned in the input text."
    )

    user_prompt = (
        f"Transform the following text into the required JSON format. "
        "Ensure that the output strictly adheres to the rules provided and includes only the information explicitly stated in the input text. "
        "If entry closing time (last entry allowed) and closing time (business end) are mentioned, clearly separate them. "
        "Add entry closing time as a 'special_condition' with 'type: \"entry_closing_time\"'. "
        "If information is missing, ambiguous, or incomplete, leave fields empty (e.g., `[]`, `{}`) and do not make assumptions:\n\n"
        f"Text: {input_text}\n\n"
        "Output example:\n"
        "{\n"
        '  "hours": {\n'
        '    "Monday": [],\n'
        '    "Tuesday": [{"start": "10:00", "end": "18:00"}],\n'
        '    "Wednesday": [{"start": "10:00", "end": "18:00"}],\n'
        '    "Thursday": [{"start": "10:00", "end": "18:00"}],\n'
        '    "Friday": [{"start": "10:00", "end": "18:00"}],\n'
        '    "Saturday": [{"start": "10:00", "end": "18:00"}],\n'
        '    "Sunday": [{"start": "10:00", "end": "18:00"}]\n'
        "  },\n"
        '  "special_conditions": [\n'
        '    {"type": "holiday", "details": "Closed on public holidays"},\n'
        '    {"type": "holiday", "details": "Closed on New Year\'s Day"},\n'
        '    {"type": "holiday", "details": "Closed on Lunar New Year and Chuseok"},\n'
        '    {"type": "entry_closing_time", "details": "Entry closes at 17:30 every day"}\n'
        "  ]\n"
        "}"
    )

    
    response = openai.ChatCompletion.create(
        # model="gpt-3.5-turbo-0125",
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0
    )
    
    return response["choices"][0]["message"]["content"]

with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

transformed_data = []

for item in tqdm(data):
    try:
        input_text = item.get("transformed_data", "")
        transformed_json = transform_data_with_gpt(input_text)  
        transformed_data.append({
            "row_index": item["row_index"],
            "transformed_data": json.loads(transformed_json)  
        })
    except Exception as e:
        print(f"Error processing row_index {item['row_index']}: {e}")

with open(output_file, "w", encoding="utf-8") as f:
    json.dump(transformed_data, f, ensure_ascii=False, indent=4)

print(f"Transformed data saved to {output_file}")


  8%|▊         | 508/6062 [12:46<4:00:27,  2.60s/it]

Error processing row_index 7507: Expecting property name enclosed in double quotes: line 10 column 3 (char 501)


 45%|████▍     | 2705/6062 [1:13:49<1:36:17,  1.72s/it]

Error processing row_index 9704: Expecting property name enclosed in double quotes: line 10 column 3 (char 321)


 45%|████▌     | 2739/6062 [1:14:47<2:12:31,  2.39s/it]

...

Error processing row_index 12118: Expecting property name enclosed in double quotes: line 10 column 3 (char 321)


 85%|████████▍ | 5123/6062 [2:20:51<34:57,  2.23s/it]

Error processing row_index 12122: Expecting property name enclosed in double quotes: line 10 column 3 (char 321)


100%|██████████| 6062/6062 [2:47:32<00:00,  1.66s/it]


Transformed data saved to transformed_tourapi_touristattractions_available_time_results_v2_2.json


In [13]:
import json
import pandas as pd

file_path1 = "/home/hwang/projects/yanolja/src/collecting_data/data_analysis/transformed_tourapi_touristattractions_available_time_results_v2_1.json"
file_path2 = "/home/hwang/projects/yanolja/src/collecting_data/data_analysis/transformed_tourapi_touristattractions_available_time_results_v2_2.json"

with open(file_path1, 'r', encoding='utf-8') as f1:
    at1 = json.load(f1)

with open(file_path2, 'r', encoding='utf-8') as f2:
    at2 = json.load(f2)

available_time1 = pd.DataFrame(at1)
available_time2 = pd.DataFrame(at2)

tmp = pd.concat([available_time1, available_time2], axis=0)

tourapi.reset_index(drop=True, inplace=True)
tmp.reset_index(drop=True, inplace=True)

tourapi = pd.concat([tourapi, tmp], axis=1)

tourapi.rename(columns={'transformed_data': 'available_time_v2'}, inplace=True)

In [14]:
tourapi[((tourapi['latitude'] <= 33) | (tourapi['latitude'] >= 43)) & ((tourapi['longitude'] <= 124) | (tourapi['longitude'] >= 132))]

,source,category,place_name,address,latitude,longitude,available_time,parking,description,zip_code,...,available_period,heritage,experience_guide,age_available,stroller,pet,credit_card,details,row_index,available_time_v2
586,tourAPI,tourist_attractions,대현산배수지공원 / 대현산배수지공원 모노레일,서울특별시 성동구 금호로 172-1 (금호동1가),19.694427,117.992566,연중개방,불가,응봉은 산 모양이 매처럼 보이고 조선시대 왕들이 매사냥을 하였다하여 한자어로 붙여진...,04721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,이용가능시설:잔디광장 / 테니스장 / 농구장 / 배드민턴장 / 야외무대 / 놀이터 ...,586.0,"{'hours': {'Monday': [], 'Tuesday': [], 'Wedne..."
1993,tourAPI,tourist_attractions,삼성해맞이공원,서울특별시 강남구 삼성동 82,19.694427,117.992566,상시 이용,불가,서울특별시 강남구 삼성동에 위치한 공원이다. 원래 상수원을 공급하기 위한 시설이었던...,6079,...,NaN,NaN,"강남유닉투어 운영 (동절기, 하절기 제외)",NaN,NaN,NaN,NaN,화장실:없음 \n입 장 료:무료 \n,1993.0,"{'hours': {'Monday': [], 'Tuesday': [], 'Wedne..."
2196,tourAPI,tourist_attractions,오산반려동물테마파크,경기도 오산시 오산천로 72오산반려동물테마파크,19.694427,117.992566,도그런<br>\n- 하절기 (6월~9월) 10:00~12:00<br> \n- 동절기...,가능,오산반려동물테마파크는 오산시에서 설립한 반려동물 복합 문화공간으로 다양한 교육 프로...,18130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"시설이용료:도그런 일일 이용권<br> \n4시간 5,000원\n",2196.0,"{'hours': {'Monday': [], 'Tuesday': [{'start':..."
8843,tourAPI,tourist_attractions,정부대전청사숲의공원,대전광역시 서구 청사로 189 (둔산동),19.694427,117.992566,제한 없음,불가,"정부대전청사숲의공원은 봄, 여름, 가을, 겨울 4계절 특성을 모두 느낄 수 있는 드...",35208,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8849.0,"{'hours': {'Monday': [], 'Tuesday': [], 'Wedne..."
9216,tourAPI,tourist_attractions,순천 오천그린광장,전라남도 순천시 오천동 702-1,0.000000,0.000000,상시개방 <br>\n※ 주차장 입차 제한시간 22:00~08:30,가능(오천그린광장 주차장 이용),오천그린광장은 국내에서 처음으로 대규모 홍수를 예방하는 기능에 머물던 저류지를 정원...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9222.0,"{'hours': {'Monday': [], 'Tuesday': [], 'Wedne..."
11039,tourAPI,tourist_attractions,DMZ 평화의 길 '테마노선',NaN,0.000000,0.000000,NaN,NaN,\n ‘평화의 길’은 전쟁의 상흔과 분단의 아픔이 서린 비무장지대와 접경지역을...,NaN,...,NaN,NaN,지역 : 강화<br>\n거리 : 61.6km<br>\n총 소요시간 : 약 5~6시간...,NaN,NaN,NaN,NaN,NaN,11051.0,"{'hours': {'Monday': [], 'Tuesday': [], 'Wedne..."


In [15]:
from dotenv import load_dotenv
import os
import numpy as np
import pandas as pd

import importlib
import analysis_utils as analysis_utils 
importlib.reload(analysis_utils)

from analysis_utils import get_geocode

load_dotenv()

client_id = os.getenv("naver_geocoding_client_id")
client_secret = os.getenv("naver_geocoding_client_secret")

# query = "서울특별시 강남구 가로수길 69 엘큐브elcube"

# geocode = get_geocode(query=query, client_id=client_id, client_secret=client_secret)
# geocode

filtered_row = tourapi[((tourapi['latitude'] <= 33) | (tourapi['latitude'] >= 43)) & ((tourapi['longitude'] <= 124) | (tourapi['longitude'] >= 132))]

for idx in filtered_row.index:
    query = tourapi.iloc[idx]['address']
    geocode = get_geocode(query=query, client_id=client_id, client_secret=client_secret)

    if geocode is None or pd.isna(geocode):
        continue

    tourapi.loc[idx, 'latitude'] = np.float64(geocode['latitude'])
    tourapi.loc[idx, 'longitude'] = np.float64(geocode['longitude'])

In [16]:
tourapi[((tourapi['latitude'] <= 33) | (tourapi['latitude'] >= 43)) & ((tourapi['longitude'] <= 124) | (tourapi['longitude'] >= 132))]

,source,category,place_name,address,latitude,longitude,available_time,parking,description,zip_code,...,available_period,heritage,experience_guide,age_available,stroller,pet,credit_card,details,row_index,available_time_v2
11039,tourAPI,tourist_attractions,DMZ 평화의 길 '테마노선',NaN,0.0,0.0,NaN,NaN,\n ‘평화의 길’은 전쟁의 상흔과 분단의 아픔이 서린 비무장지대와 접경지역을...,NaN,...,NaN,NaN,지역 : 강화<br>\n거리 : 61.6km<br>\n총 소요시간 : 약 5~6시간...,NaN,NaN,NaN,NaN,NaN,11051.0,"{'hours': {'Monday': [], 'Tuesday': [], 'Wedne..."


In [17]:
tourapi[tourapi['address'].isnull()]

,source,category,place_name,address,latitude,longitude,available_time,parking,description,zip_code,...,available_period,heritage,experience_guide,age_available,stroller,pet,credit_card,details,row_index,available_time_v2
11039,tourAPI,tourist_attractions,DMZ 평화의 길 '테마노선',NaN,0.0,0.0,NaN,NaN,\n ‘평화의 길’은 전쟁의 상흔과 분단의 아픔이 서린 비무장지대와 접경지역을...,NaN,...,NaN,NaN,지역 : 강화<br>\n거리 : 61.6km<br>\n총 소요시간 : 약 5~6시간...,NaN,NaN,NaN,NaN,NaN,11051.0,"{'hours': {'Monday': [], 'Tuesday': [], 'Wedne..."


In [19]:
from dotenv import load_dotenv
import os
import numpy as np

import importlib
import analysis_utils as analysis_utils
importlib.reload(analysis_utils)

from analysis_utils import get_reverse_geocode

load_dotenv()

client_id = os.getenv("naver_geocoding_client_id")
client_secret = os.getenv("naver_geocoding_client_secret")

# latitude = 37.472404
# longitude = 126.634438

# reverse_geocode = get_reverse_geocode(latitude=latitude, longitude=longitude, client_id=client_id, client_secret=client_secret)
# reverse_geocode

idx = 11039

latitude, longitude = tourapi.loc[idx, ['latitude', 'longitude']]
reverse_geocode = get_reverse_geocode(latitude=latitude, longitude=longitude, client_id=client_id, client_secret=client_secret)

tourapi.loc[idx, 'address'] = reverse_geocode['address'].strip()

TypeError: 'NoneType' object is not subscriptable

In [20]:
tourapi[tourapi['address'].isnull()]

,source,category,place_name,address,latitude,longitude,available_time,parking,description,zip_code,...,available_period,heritage,experience_guide,age_available,stroller,pet,credit_card,details,row_index,available_time_v2
11039,tourAPI,tourist_attractions,DMZ 평화의 길 '테마노선',NaN,0.0,0.0,NaN,NaN,\n ‘평화의 길’은 전쟁의 상흔과 분단의 아픔이 서린 비무장지대와 접경지역을...,NaN,...,NaN,NaN,지역 : 강화<br>\n거리 : 61.6km<br>\n총 소요시간 : 약 5~6시간...,NaN,NaN,NaN,NaN,NaN,11051.0,"{'hours': {'Monday': [], 'Tuesday': [], 'Wedne..."


In [21]:
tourapi = tourapi.dropna(subset=['address'])

In [22]:
tourapi[tourapi['description'].isnull()]

,source,category,place_name,address,latitude,longitude,available_time,parking,description,zip_code,...,available_period,heritage,experience_guide,age_available,stroller,pet,credit_card,details,row_index,available_time_v2
2028,tourAPI,tourist_attractions,광교호수공원 반려견 놀이터,경기도 수원시 영통구 광교호수로 214 광교호수공원시설 주차관리소광교호수공원 반려견...,37.283561,127.077264,10:00~22:00\n월요일 휴무,유료주차 1000원 (제3 주차장 3시간 이내 ),NaN,16511,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2028.0,"{'hours': {'Monday': [], 'Tuesday': [{'start':..."
2222,tourAPI,tourist_attractions,찬우물 체험동물농장,경기도 고양시 일산동구 식사로36번길 1-27 (식사동),37.668898,126.816936,10:00~18:00(3월~10월) / 10:00~17:00(11월~2월),있음(무료),NaN,10299,...,NaN,NaN,흑염소 먹이 주기 / 가을 도토리 / 밤 줍기,NaN,NaN,NaN,NaN,이용가능시설:동물관 / 조류관 / 곤충관 / 수족관 / 야외 동물원\n화장실:있음(...,2222.0,"{'hours': {'Monday': [], 'Tuesday': [{'start':..."
4033,tourAPI,tourist_attractions,춘천시 반려견놀이터 같이놀개,강원특별자치도 춘천시 영서로 3282,37.941610,127.712282,NaN,NaN,NaN,24202,...,NaN,NaN,NaN,NaN,NaN,가능,NaN,NaN,4033.0,"{'hours': {'Monday': [], 'Tuesday': [], 'Wedne..."
4420,tourAPI,tourist_attractions,할매탕,부산광역시 해운대구 중동2로10번길 7 (중동),35.163096,129.164902,매일 07:00~21:00,"가능(해운대 온천센터 주차장 공용 이용, 소형 274대, 사우나 3시간 무료)",NaN,48096,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"시설이용료:대인 9,000 / 소인 5,000원\n",4421.0,"{'hours': {'Monday': [], 'Tuesday': [], 'Wedne..."
4620,tourAPI,tourist_attractions,화양근린공원,전라남도 함평군 함평읍 내교리,35.056839,126.518112,[황금박쥐전시관] \n나비 축제 등 행사 기간에만 관람 가능,있음 <br> 주차요금 - 무료,NaN,57153,...,NaN,NaN,NaN,NaN,NaN,가능,NaN,등산로:함평 천지길(산책길)\n입 장 료:[화양근린공원] 무료\n,4621.0,"{'hours': {'Monday': [], 'Tuesday': [], 'Wedne..."
4621,tourAPI,tourist_attractions,청담도로공원,서울특별시 강남구 삼성동80,37.520694,127.062812,NaN,있음,NaN,6085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,주차요금:3시간 무료\n\n이용가능시설:팔각정\n\n화장실:있음(남녀 구분)\n\n,4622.0,"{'hours': {'Monday': [], 'Tuesday': [], 'Wedne..."
9448,tourAPI,tourist_attractions,대전반려동물공원,대전광역시 유성구 금고길 7,36.459778,127.384479,10:00~21:00,있음 (무료),NaN,34001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,화장실:있음 (남녀 구분)\n,9454.0,"{'hours': {'Monday': [], 'Tuesday': [], 'Wedne..."
9452,tourAPI,tourist_attractions,으르렁파크,경기도 안산시 단원구 대부해안로 164,37.260543,126.560412,오전반 10:00-14:00\n시설점검시간 14:00-15:00\n오후반 15:00...,주차가능,NaN,15640,...,"평일 운영 X / 토,일 (공휴일만 운영)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9458.0,"{'hours': {'Monday': [], 'Tuesday': [], 'Wedne..."
11061,tourAPI,tourist_attractions,부주산 반려동물놀이터,전라남도 목포시 녹색로 139 (옥암동)부주산 반려동물 놀이터,34.814524,126.429478,09:00~18:00,NaN,NaN,58670,...,NaN,NaN,NaN,NaN,NaN,가능,NaN,NaN,11073.0,"{'hours': {'Monday': [], 'Tuesday': [], 'Wedne..."
11066,tourAPI,tourist_attractions,의성펫월드,경상북도 의성군 단북면 안계길 255-13,36.392413,128.427791,10:00 ~ 18:00,NaN,NaN,37372,...,NaN,NaN,"어질리티(Agility) 체험 프리스비(Frisbee)체험 반려동물퀴즈대회, 반려견...",NaN,NaN,가능,NaN,NaN,11078.0,"{'hours': {'Monday': [], 'Tuesday': [], 'Wedne..."


In [23]:
tourapi[tourapi['description'].isnull()]['place_name'].tolist()

['광교호수공원 반려견 놀이터',
 '찬우물 체험동물농장',
 '춘천시 반려견놀이터 같이놀개',
 '할매탕',
 '화양근린공원',
 '청담도로공원',
 '대전반려동물공원',
 '으르렁파크',
 '부주산 반려동물놀이터',
 '의성펫월드',
 '구로구 반려견 놀이터',
 '뭐하농 하우스(농업회사법인 ㈜뭐하농)',
 '캔디원',
 '청도 와인터널']

In [24]:
tourapi.loc[tourapi['place_name'] == '광교호수공원 반려견 놀이터'] = "<반려견 놀이터>는 수원시 영통구에 위치한 광교호수공원 제3주차장 녹지공간에 조성되어 있다. 수원시의 첫 애견 놀이터인 이곳은 총 면적 3,524㎡로 전국에서 가장 큰 규모를 자랑한다. 화려한 놀이시설보다는 반려견들이 마음껏 뛰놀 수 있도록 탁 트인 공간이 매력이다. <반려견 놀이터>는 소형견 놀이터와 중·대형견 놀이터를 분리해 놓은 것이 특징이다. 여기에 흥분한 개를 진정시키기 위한 안전 격리 공간까지 별도로 마련되어 있다."
tourapi.loc[tourapi['place_name'] == '찬우물 체험동물농장'] = "찬우물 아기동물원은 11,000여 평의 숲속에 동물관, 조류관, 수족관, 곤충관 등을 갖추고 어린이 여러분의 방문을 기다리고 있다. 이곳에 오면 동물들을 가까이서 보고 같이 노는 체험뿐만 아니라 먹이주기, 나무곤충만들기 등 다양하게 마련된 프로그램을 즐길 수 있다. 그리고 버섯농장에서는 표고버섯을 직접 따서 부침개도 해먹을 수 있다.(단체일 경우만)"
tourapi.loc[tourapi['place_name'] == '춘천시 반려견놀이터 같이놀개'] = "같이놀개'는 춘천시 동물보호센터 내에 위치한 반려견 놀이터로, 동물등록을 한 반려견과 보호자가 무료로 이용할 수 있다. 질병이 있는 반려견이나 발정이 있는 반려견, 맹견 등은 출입이 제한된다. 반려견 놀이터 내에는 어질리티 기구, 음수전, 배변봉투함 등이 조성되어 있으며, 이용자는 입장 전 출입 명부 작성 후 반려견과 함께 자유롭게 이용할 수 있다."
tourapi.loc[tourapi['place_name'] == '할매탕'] = "할매탕은 1935년 해운대 최초의 대중목욕탕으로, 피부병에 효과가 있는 온천수로 유명했다. 90년 전통의 원탕 온천으로, 피부 미용, 관절염, 아토피 등에 효과적인 성분이 가득한 온천수를 제공한다."
tourapi.loc[tourapi['place_name'] == '화양근린공원'] = "화양근린공원은 전라남도 함평군 함평읍 내교리에 위치한 공원으로, 함평의 엑스포공원으로도 불린다. 아담한 공원길과 탁 트인 함평의 전망을 볼 수 있으며, 느긋하게 산책을 즐기기 좋은 곳이다."
tourapi.loc[tourapi['place_name'] == '청담도로공원'] = "잠실대교와 청담대교 사이 올림픽대로에 위치한 청담도로공원. 1987년 올림픽대로 준공을 기념하기 위해 조성된 공원이다. 뜻밖의 장소에서 만나는 우연의 장소, 바로 청담도로공원이다. 공원길에 연결되어 있는 지하통로를 따라 내려가면 청담나들목 중간부분과 이어진다."
tourapi.loc[tourapi['place_name'] == '대전반려동물공원'] = "그중 대전시 유성구 금고동에 위치하고 있는 대전반려동물공원은 2022년 6월에 정식 개장한 곳으로 약 3만 m² 넓은 부지 위에 반려동물 문화센터와 유기 동물 보호소를 함께 운영하고 있다. 이곳은 천연 잔디로 조성된 야외 놀이터가 매력적인 곳으로, 대·중·소형견 동물 놀이터, 야외훈련장, 산책로뿐만 아니라 야간에도 편하게 이용 가능한 산책로가 잘 조성되어 있다. 22년 7월에는 보호자와 반려견이 함께 이용할 수 있는 야외 수영장이 오픈하면서 평소보다 더 많은 반려인들이 방문했다."
# 으르렁파크 - 정보 없음
# 부주산 반려동물놀이터 - 정보 없음
tourapi.loc[tourapi['place_name'] == '의성펫월드'] = "의성 단북면 안계길 일원에 지난 2020년 개장한 의성 펫월드는 반려견 테마파크이다. 오롯이 반려인과 반려동물을 위한 공간이다. 축구장 6개 크기에 달하는 약 1만 2천평 부지에 반려동물과 보호자가 함께 즐길 수 있는 다양한 놀이시설과 편의시설을 갖추었다."
tourapi.loc[tourapi['place_name'] == '구로구 반려견 놀이터'] = "구로구에서는 안양천 고척교 아래에 반려견 놀이터를 개장했다. 서울시 최대 규모의 반려견 놀이터다. 안양천 인근은 공기도 맑고 공터를 적절하게 활용할 수 있어서 반려견 놀이터로서는 적격이다. 놀이터의 규모는 2,600㎡(약 800평)이다."
tourapi.loc[tourapi['place_name'] == '뭐하농 하우스(농업회사법인 ㈜뭐하농)'] = "'뭐하농'은 지난해 2월 괴산지역 20~30대 청년 농부 여섯이 '농부들이 생산해낸 생산물과 농부의 가치가 멋있게 활용될 수 있도록 농업 콘텐츠를 만들어보자'며 결성한 주식회사다. 귀농 부부 2쌍과 아버지를 따라 농사를 짓는 청년 2명이 모였다."
tourapi.loc[tourapi['place_name'] == '캔디원'] = "캔디원은 조천읍에 있는 수제캔디 전문점으로 [SWEET TO EVERYONE(모두에게 달콤한)]을 모토로 수제캔디를 통해 달콤한 행복을 주기 위해 2014년 문을 열었다. 수제캔디 전문점으로는 세계에서 가장 큰 규모이이다. 오래전부터 사용하던 사탕기계와 다양한 사진전시물을 통해 사탕의 유래를 한눈에 볼 수 있고, 전 세계 다양하고 신기한 사탕들이 전시되어 있어 사탕에 관한 신비한 볼거리가 많다. 아이들이 체험을 즐길 수 있도록 캔디체험학습장을 보유하여, 만들어진 수제캔디를 직접 잘라보고, 다양한 색의 롤리팝 캔디를 우리 아이들의 손으로 직접 만들어보는 체험을 즐길 수 있다. 캔디열판이 뜨겁기 때문에 캔디만들기 체험은 기본적으로 키가 120cm 이상이 되어야 안전하게 체험할 수 있으며 키가 되지 않은 5살 이상 미취학 아동은 부모와 함께 캔디 컷팅체험을 할 수 있다. 판매장에서는 시중에서는 볼 수 없는 다양한 모양의 수제캔디와 젤리, 그 외에도 선물이나 기념품으로 구매할 만한 굿즈들을 구매할 수 있다."
tourapi.loc[tourapi['place_name'] == '청도 와인터널'] = "청도 와인터널은 경상북도 청도군 화양읍에 있는 옛 경부선 열차 터널을 개조하여 만든 와인 숙성고이자 관광지이다. 1905년에 개통된 터널을 정비하여 2006년 3월에 개장했으며, 와인 숙성에 적합한 온도와 습도를 유지하고 있다."


/tmp/ipykernel_654927/3071097200.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '<반려견 놀이터>는 수원시 영통구에 위치한 광교호수공원 제3주차장 녹지공간에 조성되어 있다. 수원시의 첫 애견 놀이터인 이곳은 총 면적 3,524㎡로 전국에서 가장 큰 규모를 자랑한다. 화려한 놀이시설보다는 반려견들이 마음껏 뛰놀 수 있도록 탁 트인 공간이 매력이다. <반려견 놀이터>는 소형견 놀이터와 중·대형견 놀이터를 분리해 놓은 것이 특징이다. 여기에 흥분한 개를 진정시키기 위한 안전 격리 공간까지 별도로 마련되어 있다.' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tourapi.loc[tourapi['place_name'] == '광교호수공원 반려견 놀이터'] = "<반려견 놀이터>는 수원시 영통구에 위치한 광교호수공원 제3주차장 녹지공간에 조성되어 있다. 수원시의 첫 애견 놀이터인 이곳은 총 면적 3,524㎡로 전국에서 가장 큰 규모를 자랑한다. 화려한 놀이시설보다는 반려견들이 마음껏 뛰놀 수 있도록 탁 트인 공간이 매력이다. <반려견 놀이터>는 소형견 놀이터와 중·대형견 놀이터를 분리해 놓은 것이 특징이다. 여기에 흥분한 개를 진정시키기 위한 안전 격리 공간까지 별도로 마련되어 있다."
/tmp/ipykernel_654927/3071097200.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '<반려견 놀이터>는 수원시 영통구에 위치한 광교호수공원 

In [25]:
import importlib
import analysis_utils as analysis_utils
importlib.reload(analysis_utils)

from analysis_utils import get_wikipedia_summary

tourist_spot = "잠월미술관"
summary = get_wikipedia_summary(tourist_spot)


In [26]:
tourapi[tourapi['parking'].isnull()]

,source,category,place_name,address,latitude,longitude,available_time,parking,description,zip_code,...,available_period,heritage,experience_guide,age_available,stroller,pet,credit_card,details,row_index,available_time_v2
6,tourAPI,tourist_attractions,보현사(강릉),강원특별자치도 강릉시 성산면 보현길 396,37.736626,128.769306,상시 개방,NaN,보현사는 대한불교조계종 제4교구 본사 월정사의 말사이다. 보현사는 대관령과 선자령 ...,25442,...,NaN,NaN,템플스테이(휴식형 / 체험형)<br>\n※ 자세한 사항은 홈페이지 참조,NaN,없음,NaN,NaN,화장실:있음\n,6.0,"{'hours': {'Monday': [], 'Tuesday': [], 'Wedne..."
8,tourAPI,tourist_attractions,진천 농다리,충청북도 진천군 문백면 구곡리,36.826491,127.492954,NaN,NaN,"진천 농다리는 사력 암질의 붉은 돌을 쌓아서 만들어진 다리로, 28칸의 교각이다. ...",27863,...,NaN,NaN,NaN,NaN,없음,NaN,NaN,"이용가능시설:농다리 전시관, 공원, 인공폭포, 농암정, 천년정, 수변데크 등\n화장...",8.0,"{'hours': {'Monday': [], 'Tuesday': [], 'Wedne..."
22,tourAPI,tourist_attractions,진남교반,경상북도 문경시 마성면 신현리,36.662074,128.126468,NaN,NaN,"문경은 경상북도 북쪽의 울타리이다. 태백산에서 이어진 대미산, 주흘산, 희양산 같은...",36923,...,NaN,NaN,NaN,NaN,없음,NaN,NaN,NaN,22.0,"{'hours': {'Monday': [], 'Tuesday': [], 'Wedne..."
23,tourAPI,tourist_attractions,해운대 달맞이길,부산광역시 해운대구 달맞이길 190,35.157719,129.18257,NaN,NaN,해운대는 부산 해운대구 동쪽 바닷가에 있는 동백(冬柏) 섬을 지칭하지만 오늘날에는 ...,48116,...,NaN,NaN,NaN,NaN,불가,NaN,NaN,NaN,23.0,"{'hours': {'Monday': [], 'Tuesday': [], 'Wedne..."
29,tourAPI,tourist_attractions,성철대종사생가(겁외사),경상남도 산청군 단성면 성철로 125,35.279239,127.968434,NaN,NaN,[산은 산이요 물은 물이로다]라는 법어집과 불경국역집을 남긴 성철 스님은 승려는 수...,52250,...,NaN,NaN,NaN,NaN,없음,NaN,NaN,NaN,29.0,"{'hours': {'Monday': [], 'Tuesday': [], 'Wedne..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13055,tourAPI,tourist_attractions,김대기가옥,경상북도 예천군 용문면 금당실길 80-8,36.692734,128.41025,NaN,NaN,"고즈넉한 풍경 속, 시간이 멈춰버린 듯한 금당실전통마을 초이벵 위치한 김대기 가옥은...",36835,...,NaN,NaN,NaN,NaN,없음,없음,NaN,NaN,NaN,NaN
13057,tourAPI,tourist_attractions,순천업사이클센터더새롬,전라남도 순천시 팔마1길 9-19,34.932529,127.51265,"매일 09:00~18:00, 공휴일 휴무",NaN,'새활용(Up-cycling)'이란 단순 재활용을 넘어 쓰임이 다한 자원에 디자인을...,58001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13058,tourAPI,tourist_attractions,병암 화강암 단애,경상북도 청송군 얼음골로 188-1,36.312635,129.11644,NaN,NaN,병풍을 친 것처럼 바위가 늘어서 있어 병풍바위(병암)으로 불리는 병암 화강암 단애는...,37444,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,한국어 안내서비스:5인 이상시 해설예약 가능\n,NaN,NaN
13059,tourAPI,tourist_attractions,옹장굴,경기도 포천시 윗찬물길 379-71,38.136138,127.27052,NaN,NaN,옹장굴은 중생대 쥬라기 대보화강암이 풍화되어 형서된 쇄설성 퇴적층 위에 신생대 제4...,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
import pandas as pd

parking = pd.read_excel("/home/hwang/projects/yanolja/data/dataset/etc/전국주차장정보표준데이터-20241210.xls")
parking.columns = parking.iloc[0]
parking = parking[1:].reset_index(drop=True)
parking['위도'] = pd.to_numeric(parking['위도'], errors="coerce")
parking['경도'] = pd.to_numeric(parking['경도'], errors="coerce")
parking = parking.dropna(subset=['위도', '경도'])

In [28]:
parking.head(2)

,주차장관리번호,주차장명,주차장구분,주차장유형,소재지도로명주소,소재지지번주소,주차구획수,급지구분,부제시행구분,운영요일,...,결제방법,특기사항,관리기관명,전화번호,위도,경도,장애인전용주차구역보유여부,데이터기준일자,제공기관코드,제공기관명
0,204-2-000023,아래숯골,공영,노외,경기도 성남시 수정구 성남대로 1259,NaN,150,1,요일제,평일+토요일+공휴일,...,신용카드,경차+장애인 차량 50프로 할인 등,성남도시개발공사 노외주차처,NaN,37.442288,127.126662,Y,2024-05-31,3780000,경기도 성남시
1,204-2-000024,태평4동,공영,노외,경기도 성남시 수정구 태평로 109,NaN,89,1,요일제,평일+토요일+공휴일,...,신용카드,경차+장애인 차량 50프로 할인 등,성남도시개발공사 노외주차처,NaN,37.451234,127.138580,Y,2024-05-31,3780000,경기도 성남시


In [29]:
import importlib
import analysis_utils as analysis_utils
importlib.reload(analysis_utils)

from analysis_utils import find_nearest_parking

nearest_parking = find_nearest_parking(poi_lat=37.610198, poi_lon=127.040989, parking_data=parking)

In [30]:
nearest_parking

'동방고개 공영 주차장'

In [31]:
import pandas as pd
import numpy as np

def combine_columns(row):
    additional_info = []

    # heritage 추가
    if pd.notna(row['heritage']):
        additional_info.append(f"유산구분: {row['heritage']}")

    # experience_guide 추가
    if pd.notna(row['experience_guide']):
        additional_info.append(f"체험안내: {row['experience_guide']}")

    # age_available 추가
    if pd.notna(row['age_available']):
        additional_info.append(f"체험가능연령: {row['age_available']}")
    
    # stroller 추가
    if pd.notna(row['stroller']):
        additional_info.append(f"유모차: {row['stroller']}")
    
    # pet 추가
    if pd.notna(row['pet']):
        additional_info.append(f"반려동물: {row['pet']}")
    
    details = row['details'] if pd.notna(row['details']) else ""
    
    combined_details = details + ('\n' if details and additional_info else '') + '\n'.join(additional_info)
    return combined_details

tourapi['details'] = tourapi.apply(combine_columns, axis=1)

tourapi = tourapi.drop(columns=['heritage', 'experience_guide', 'age_available', 'stroller', 'pet'])


/tmp/ipykernel_654927/1916423740.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tourapi['details'] = tourapi.apply(combine_columns, axis=1)


In [32]:
tourapi['source'] = 'tourAPI'
tourapi['category'] = 'tourist_attractions'

In [33]:
tourapi = tourapi[['source', 'category', 'place_name', 'address', 'latitude', 'longitude', 'available_time', 'available_time_v2', 'parking', 'description', 'details']]

In [34]:
tourapi

,source,category,place_name,address,latitude,longitude,available_time,available_time_v2,parking,description,details
0,tourAPI,tourist_attractions,수성못 유원지,대구광역시 수성구 용학로 35-5,35.826842,128.612873,00:00~24:00<br>\r\n※ 수성 미디어아트 음악분수 운영시간은 홈페이지 ...,"{'hours': {'Monday': [], 'Tuesday': [{'start':...",가능,수성유원지는 범물동 용지봉(629ｍ)에서 북서부로 뻗어 내린 줄기의 하부에 위치하여...,"이용가능시설:- 수성못, 오리배&유람선, 수변데크로드, 포켓무대, 관광안내소 북카페..."
1,tourAPI,tourist_attractions,동락공원,경상북도 구미시 3공단1로 191(진평동),36.097116,128.401951,상시개방,"{'hours': {'Monday': [], 'Tuesday': [], 'Wedne...",있음(약 소형 946대),낙동강을 따라 구미대교 아래 임수동에서 칠곡군 석적읍 중리까지 이르는 수변형 도시공...,주차요금:무료\n입 장 료:무료\n화장실:있음(남/녀 구분)\n이용가능시설:* 체육...
2,tourAPI,tourist_attractions,연천 초성김치마을,경기도 연천군 청산면 평화로 209,37.992731,127.069412,[체험마을] <br>\n- 10:00~16:00 <br>\n[펜션] <br>\n- ...,"{'hours': {'Monday': [], 'Tuesday': [{'start':...",가능<br>요금 (무료),경원선 초성리역∙열두개울 부근에 있는 연천 초성김치마을은 김치를 테마로 한 체험 마...,이용가능시설:김치체험장 / 인성교육실 / 세미나실 / 캠프파이어장 / 운동장 / 야...
3,tourAPI,tourist_attractions,명성산,경기도 포천시 이동면 도평리,38.095674,127.350422,NaN,"{'hours': {'Monday': [], 'Tuesday': [], 'Wedne...",있음,"가을철이면 억새산행 대상지로 유명한 산이다. 서울에서 동북으로 84km, 운천에서 ...",등산로:[탐방코스]<br />\n * 제1코스 : 상동주차장 > 비선폭포 > 등룡폭...
4,tourAPI,tourist_attractions,포천 산정호수,경기도 포천시 영북면 산정호수로411번길 104,38.068838,127.322103,상시개방,"{'hours': {'Monday': [], 'Tuesday': [], 'Wedne...","가능<br>요금(소형1일-2,000원 / 중형1일-5,000원 / 대형1일-10,0...",포천시의 북쪽 강원도 철원군과 도계에 있는 명성산(해발 923m)이 병풍처럼 둘러있...,야간산행 제한안내:탐방객 안전과 산불방지 등 공원보호를 위해 야간산행(일몰 후부터 ...
...,...,...,...,...,...,...,...,...,...,...,...
13057,tourAPI,tourist_attractions,순천업사이클센터더새롬,전라남도 순천시 팔마1길 9-19,34.932529,127.51265,"매일 09:00~18:00, 공휴일 휴무",NaN,NaN,'새활용(Up-cycling)'이란 단순 재활용을 넘어 쓰임이 다한 자원에 디자인을...,
13058,tourAPI,tourist_attractions,병암 화강암 단애,경상북도 청송군 얼음골로 188-1,36.312635,129.11644,NaN,NaN,NaN,병풍을 친 것처럼 바위가 늘어서 있어 병풍바위(병암)으로 불리는 병암 화강암 단애는...,한국어 안내서비스:5인 이상시 해설예약 가능\n
13059,tourAPI,tourist_attractions,옹장굴,경기도 포천시 윗찬물길 379-71,38.136138,127.27052,NaN,NaN,NaN,옹장굴은 중생대 쥬라기 대보화강암이 풍화되어 형서된 쇄설성 퇴적층 위에 신생대 제4...,
13060,tourAPI,tourist_attractions,오프컬리,서울특별시 성동구 서울숲2길 16-9,37.547069,127.040374,12:00 ~ 21:00,NaN,NaN,"마켓컬리로 유명한 주식회사 컬리의 첫 오프라인샵, 오프컬리이다. 1층은 예약 없이 ...",
